<a href="https://colab.research.google.com/github/Kaidavid/NLI/blob/main/NLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q datasets transformers
! pip install sentencepiece

     |████████████████████████████████| 441 kB 5.2 MB/s 
     |████████████████████████████████| 5.5 MB 55.5 MB/s 
     |████████████████████████████████| 115 kB 70.4 MB/s 
     |████████████████████████████████| 95 kB 3.6 MB/s 
     |████████████████████████████████| 212 kB 67.4 MB/s 
     |████████████████████████████████| 163 kB 66.4 MB/s 
     |████████████████████████████████| 127 kB 51.3 MB/s 
     |████████████████████████████████| 7.6 MB 54.5 MB/s 
     |████████████████████████████████| 115 kB 66.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.7 MB/s 


In [ ]:
import time
import math
import torch
import pandas as pd
import torch.nn as nn
from transformers import *
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

In [ ]:
# KLUE NLI dataset
from datasets import load_dataset
from easydict import EasyDict as edict

klue_nli = load_dataset('klue', 'nli')

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
df = pd.concat([pd.DataFrame(klue_nli['train']), pd.DataFrame(klue_nli['validation'])], axis=0)
df_source_domain = df[df['source'] != 'airbnb']
df_target_domain = df[df['source'] == 'airbnb']

print(f"Total data: {df.shape[0]} \nSource domain: {df_source_domain.shape[0]} \nTarget domain: {df_target_domain.shape[0]}")

Total data: 27998 
Source domain: 22574 
Target domain: 5424


In [ ]:
class MyDataset(Dataset):
  def __init__(self, source, target):
    self.X = source.to_dict('records')
    self.Y = target.to_dict('records')

  def __getitem__(self, index):
    if index == 'train':
      return self.X
    elif index == 'validation':
      return self.Y
    else:
      return "Wrong Input"

  def __len__(self):
    return 2

klue_nli_new = MyDataset(df_source_domain, df_target_domain)

def sample_data(how_many):
  print("Given Data")
  for i in range(how_many):
    print(klue_nli['validation'][i])

  print("\nExtract 'source': 'aribnb' for validation")
  for i in range(how_many):
    print(klue_nli_new['validation'][i])

sample_data(10)

Given Data
{'guid': 'klue-nli-v1_dev_00000', 'source': 'airbnb', 'premise': '흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.', 'hypothesis': '어떤 방에서도 흡연은 금지됩니다.', 'label': 2}
{'guid': 'klue-nli-v1_dev_00001', 'source': 'airbnb', 'premise': '10명이 함께 사용하기 불편함없이 만족했다.', 'hypothesis': '10명이 함께 사용하기 불편함이 많았다.', 'label': 2}
{'guid': 'klue-nli-v1_dev_00002', 'source': 'airbnb', 'premise': '10명이 함께 사용하기 불편함없이 만족했다.', 'hypothesis': '성인 10명이 함께 사용하기 불편함없이 없었다.', 'label': 1}
{'guid': 'klue-nli-v1_dev_00003', 'source': 'airbnb', 'premise': '10명이 함께 사용하기 불편함없이 만족했다.', 'hypothesis': '10명이 함께 사용하기에 만족스러웠다.', 'label': 0}
{'guid': 'klue-nli-v1_dev_00004', 'source': 'airbnb', 'premise': '10층에 건물사람들만 이용하는 수영장과 썬베드들이 있구요.', 'hypothesis': '건물사람들은 수영장과 썬베드를 이용할 수 있습니다.', 'label': 0}
{'guid': 'klue-nli-v1_dev_00005', 'source': 'airbnb', 'premise': '10층에 건물사람들만 이용하는 수영장과 썬베드들이 있구요.', 'hypothesis': '수영장과 썬베드는 9층에 있습니다.', 'label': 2}
{'guid': 'klue-nli-v1_dev_00006', 'source': 'airbnb', 'premise': '10층에 건물사람들만 이용하는 수영장과 썬베

In [ ]:
import os
from google.colab import drive
drive.mount('/gdrive')
root = '/gdrive/My Drive/Supercoder'
os.chdir(root)

Mounted at /gdrive


In [ ]:
torch.manual_seed(470)
torch.cuda.manual_seed(470)

args = edict()

args.batch_size = 32
args.epoch = 400

train_dataloader = DataLoader(klue_nli_new['train'], batch_size=args.batch_size, shuffle=True)
val_dataloader = DataLoader(klue_nli_new['validation'], batch_size=args.batch_size, shuffle=True)

In [ ]:
def truncation(tokenizer, sent1, sent2, max_length):
  ids_sent1 = tokenizer.encode(sent1)[1:-1]
  ids_sent2 = tokenizer.encode(sent2)[1:-1]
  half = max_length // 2

  if len(ids_sent1) > half-2 and len(ids_sent2) < half-2 and len(ids_sent1) + len(ids_sent2) > max_length-4 :
    ids_sent1 = ids_sent1[: max_length-4-len(ids_sent2)]

  elif len(ids_sent1) < half-2 and len(ids_sent2) > half-2 and len(ids_sent1) + len(ids_sent2) > max_length-4 :
    ids_sent2 = ids_sent2[: max_length-4-len(ids_sent1)]
 
  elif len(ids_sent1) > half-2 and len(ids_sent2) > half-2 :
    ids_sent1 = ids_sent1[: half-2]
    ids_sent2 = ids_sent2[: half-2]

  return tokenizer.decode(ids_sent1), tokenizer.decode(ids_sent2)

In [ ]:
def get_pair_input(tokenizer, sent1, sent2, model_type, max_length):

  sent1, sent2 = truncation(tokenizer, sent1, sent2, max_length)

  if 'roberta' in model_type:
    text = "<s> {} </s></s> {} </s>".format(sent1, sent2)
  else:
    text = "[CLS] {} [SEP] {} [SEP]".format(sent1, sent2)

  tokenized_text = tokenizer.tokenize(text)
  indexed_tokens = tokenizer.encode(text)[1:-1]
  assert len(tokenized_text) == len(indexed_tokens)

  # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
  segments_ids = []
  sep_flag = False

  for i in range(len(indexed_tokens)):
    if 'roberta' in model_type and tokenized_text[i] == '</s>' and not sep_flag:
      segments_ids.append(0)
      sep_flag = True
    elif 'bert-' in model_type and tokenized_text[i] == '[SEP]' and not sep_flag:
      segments_ids.append(0)
      sep_flag = True
    elif sep_flag:
      segments_ids.append(1)
    else:
      segments_ids.append(0)
  return indexed_tokens, segments_ids

In [ ]:
def build_batch(tokenizer, sent_pair_list, model_type, max_length):
  token_id_list = []
  segment_list = []
  attention_masks = []

  for pair in sent_pair_list:
    sent1, sent2 = pair 
    ids, segs = get_pair_input(tokenizer, sent1, sent2, model_type, max_length)
    
    token_id_list.append(ids)
    segment_list.append(segs)
    attention_masks.append([1]*len(ids))

  if len(token_id_list) == 0:
    return None, None, None

  # PADDING
  assert(len(token_id_list) == len(segment_list))
  
  for ii in range(len(token_id_list)):
    token_id_list[ii] += [0]*(max_length-len(token_id_list[ii]))
    segment_list[ii] += [1]*(max_length-len(segment_list[ii]))
    attention_masks[ii] += [0]*(max_length-len(attention_masks[ii]))

  return token_id_list, segment_list, attention_masks

In [ ]:
class BERTNLIModel(nn.Module):
  def __init__(self, bert_type='bert-base-multilingual-cased', label_num=3, max_length=512):
    super(BERTNLIModel, self).__init__() 
    self.bert_type = bert_type
    self.max_length = max_length

    if 'bert-base-uncased' in bert_type:
      self.bert = BertModel.from_pretrained(bert_type)
      self.tokenizer = BertTokenizer.from_pretrained(bert_type)
      
    elif 'bert-large' in bert_type:
      self.bert = BertModel.from_pretrained(bert_type)
      self.tokenizer = BertTokenizer.from_pretrained(bert_type)
  
    elif 'roberta' in bert_type:
      self.bert = AutoModelForMaskedLM.from_pretrained(bert_type)
      self.tokenizer = AutoTokenizer.from_pretrained(bert_type)

    elif 'multilingual' in bert_type:
      config = AutoConfig.from_pretrained(bert_type, num_labels=label_num, output_hidden_states=True)
      self.bert = AutoModelForSequenceClassification.from_pretrained(bert_type, config=config)
      self.tokenizer = AutoTokenizer.from_pretrained(bert_type)

    elif 'albert' in bert_type:
      self.bert = AlbertModel.from_pretrained(bert_type)
      self.tokenizer = AlbertTokenizer.from_pretrained(bert_type)

    else:
      print('illegal bert type {}!'.format(bert_type))

    self.nli_head = nn.Linear(self.bert.config.hidden_size, 128)
    self.nli_tail = nn.Linear(128, label_num)
    self.bn = nn.BatchNorm1d(128)
    self.act = nn.ReLU()
    self.dp = nn.Dropout(0.2)


  def forward(self, sent_pair_list, method):
    ids, types, masks = build_batch(self.tokenizer, sent_pair_list, self.bert_type, self.max_length)

    if ids is None: return None

    ids_tensor = torch.tensor(ids).cuda()
    types_tensor = torch.tensor(types).cuda()
    masks_tensor = torch.tensor(masks).cuda()

    assert ids_tensor.size() == types_tensor.size() == masks_tensor.size()
    
    hidden_states = self.bert(input_ids=ids_tensor, token_type_ids=types_tensor, attention_mask=masks_tensor).hidden_states
    last_hidden_state = hidden_states[-1]

    if method == 0:
      output = last_hidden_state[:, 0, :]

    else:
      mask_expanded = masks_tensor.unsqueeze(-1).expand(last_hidden_state.size())      
      # Compute embedding of sentence from hidden state of each token in the sentence
      output = torch.sum(last_hidden_state * mask_expanded, 1) / torch.clamp(mask_expanded.sum(1), min=1e-9)

    output = self.dp(self.act(self.bn(self.nli_head(output))))
    output = self.nli_tail(output)

    return output

In [ ]:
#  method=0: takes embedding of the first token, method=1: averages embeddings of all tokens from the last hidden state
method = 0  # to change the way the last hidden state is used ----> method=1: performs worse
ckpt_dir = Path(root + f'/ckpt_{method}')
ckpt_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
def train(model, optimizer, scheduler, criterion, model_type, method=0):
  best_accuracy = 0
  ep = 0
  training_loss, training_accuracy = [], []
  validation_loss, validation_accuracy = [], []

  if len(os.listdir(ckpt_dir)) != 0:
    try:
      checkpoint = torch.load(f'{ckpt_dir}/{model_type}_best.pt')
      model.load_state_dict(checkpoint['model_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      ep = checkpoint['epoch'] + 1
      best_accuracy = checkpoint['best_accuracy']
      print(f"Resuming from previous episode: {ep}, best_accuracy: {best_accuracy}")
    except:
      print(f"Something wrong with {ckpt_dir}/{model_type}_best.pt model. Please remove it!")

  for epoch in range(ep, args.epoch):
    # Here starts the train loop.
    model.train()

    train_loss = 0.
    train_accuracy = 0.
    train_num_data = 0.

    for batch_idx, data in tqdm(enumerate(train_dataloader)):
      sent_1 = data['premise']
      sent_2 = data['hypothesis']
      labels = data['label'].cuda()

      logit = model(zip(sent_1, sent_2), method)

      # Compute accuracy of this batch using `logit`, and keep it in a variable called 'accuracy'.
      accuracy = (logit.argmax(1) == labels).float().mean()

      # Compute loss using `logit` and `y`, and keep it in a variable called `loss`.
      loss = criterion(logit, labels)

      # flush out the previously computed gradient.
      optimizer.zero_grad()

      # backward the computed loss.
      loss.backward()

      # update the network weights.
      optimizer.step()

      train_loss += loss.item()*len(labels)
      train_accuracy += accuracy.item()*len(labels)
      train_num_data += len(labels)

    train_loss /= train_num_data
    train_accuracy /= train_num_data

    training_loss.append(train_loss)
    training_accuracy.append(train_accuracy)

    print(f'Training result {epoch}/{args.epoch} || loss : {train_loss:.3f} acc : {train_accuracy:.3f} ')

    # Here starts the validation loop.
    model.eval()
    with torch.no_grad():
      val_loss = 0.
      val_accuracy = 0.
      val_num_data = 0.
      for batch_idx, data in tqdm(enumerate(val_dataloader)):
        sent_1 = data['premise']
        sent_2 = data['hypothesis']
        labels = data['label'].cuda()

        logit = model(zip(sent_1, sent_2), method)

        # Compute loss using `logit` and `y`, and keep it in a variable called `loss`.
        loss = criterion(logit, labels)

        # Compute accuracy of this batch using `logit`, and keep it in a variable called 'accuracy'.
        accuracy = (logit.argmax(dim=1) == labels).float().mean()

        val_loss += loss.item()*len(labels)
        val_accuracy += accuracy.item()*len(labels)
        val_num_data += len(labels)

      val_loss /= val_num_data
      val_accuracy /= val_num_data

      validation_loss.append(val_loss)
      validation_accuracy.append(val_accuracy)

      print(f'Validation result {epoch}/{args.epoch} || loss : {val_loss:.3f} acc : {val_accuracy:.3f} ')

      # Whenever `test_accuracy` is greater than `best_accuracy`, save network weights with the filename 'best.pt' in the directory specified by `ckpt_dir`.
      if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(
            {
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'best_accuracy': best_accuracy
            },
            f'{ckpt_dir}/{model_type}_best.pt')

    scheduler.step()

  return best_accuracy, training_loss, training_accuracy, validation_loss, validation_accuracy

In [ ]:
def get_scheduler(optimizer, scheduler: str, warmup_steps: int, t_total: int):
  scheduler = scheduler.lower()
  if scheduler=='constantlr':
    return get_constant_schedule(optimizer)
  elif scheduler=='warmupconstant':
    return get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)
  elif scheduler=='warmuplinear':
    return get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
  elif scheduler=='warmupcosine':
    return get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
  elif scheduler=='warmupcosinewithhardrestarts':
    return get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
  else:
    raise ValueError("Unknown scheduler {}".format(scheduler))

In [ ]:
types = ['bert-base-uncased', 'bert-large-uncased', "xlm-roberta-base", 'bert-base-multilingual-cased', 'albert-base-v2']
model = BERTNLIModel(bert_type='bert-base-multilingual-cased', label_num=3, max_length=128).cuda()

optimizer = AdamW(model.parameters(),lr=2e-5,eps=1e-6,correct_bias=False)
criterion = nn.CrossEntropyLoss()

warmup_percent = 0.2
total_steps = math.ceil(args.epoch*len(klue_nli['train'])*1./args.batch_size)
warmup_steps = int(total_steps*0.2)
scheduler = get_scheduler(optimizer, 'WarmupLinear', warmup_steps=warmup_steps, t_total=total_steps)

start_time = time.time()
best_accuracy, training_loss, training_accuracy, validation_loss, validation_accuracy = train(model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion, model_type=types[3], method=method)
duration = time.time() - start_time
print(f'\n\n\n Best accuracy : {best_accuracy:.3f} took {duration:.3f} secs')

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head"

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/pytorch_model.bin
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly ide

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

...

706it [08:18,  1.42it/s]


Training result 79/400 || loss : 1.072 acc : 0.440 


170it [00:41,  4.10it/s]


Validation result 79/400 || loss : 1.069 acc : 0.431 


238it [02:48,  1.42it/s]

## **Report**
<strong>
<ol>
  <li>Task 1</li>
  <ul>
    <li>Best accuracy = 0.673, Epoch = 124</li>
    <li>Training can always be resumed</li>
  </ul>
</ol>
</strong>